In [ ]:
# default_exp models.hofm

# HOFM
> A pytorch implementation of Higher-Order Factorization Machines.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

## v1

In [ ]:
#export
import torch
from torch import nn

from itertools import combinations

from recohut.models.layers.embedding import EmbeddingLayer
from recohut.models.layers.interaction import InnerProductLayer
from recohut.models.layers.common import LR_Layer

from recohut.models.bases.ctr import CTRModel

In [ ]:
#export
class HOFM(CTRModel):
    def __init__(self, 
                 feature_map, 
                 model_id="HOFM",
                 task="binary_classification",
                 learning_rate=1e-3, 
                 embedding_initializer="torch.nn.init.normal_(std=1e-4)",
                 order=3,
                 embedding_dim=10,
                 reuse_embedding=False,
                 **kwargs):
        super(HOFM, self).__init__(feature_map, 
                                           model_id=model_id,
                                           **kwargs)
        self.order = order
        assert order >= 2, "order >= 2 is required in HOFM!"
        self.reuse_embedding = reuse_embedding
        if reuse_embedding:
            assert isinstance(embedding_dim, int), "embedding_dim should be an integer when reuse_embedding=True."
            self.embedding_layer = EmbeddingLayer(feature_map, embedding_dim)
        else:
            if not isinstance(embedding_dim, list):
                embedding_dim = [embedding_dim] * (order - 1)
            self.embedding_layers = nn.ModuleList([EmbeddingLayer(feature_map, embedding_dim[i]) \
                                                   for i in range(order - 1)])
        self.inner_product_layer = InnerProductLayer(feature_map.num_fields)
        self.lr_layer = LR_Layer(feature_map, use_bias=True)
        self.output_activation = self.get_final_activation(task)
        self.field_conjunction_dict = dict()
        for order_i in range(3, self.order + 1):
            order_i_conjunction = zip(*list(combinations(range(feature_map.num_fields), order_i)))
            for k, field_index in enumerate(order_i_conjunction):
                self.field_conjunction_dict[(order_i, k)] = torch.LongTensor(field_index)
        self.init_weights(embedding_initializer=embedding_initializer)

    def forward(self, inputs):
        y_pred = self.lr_layer(inputs)
        if self.reuse_embedding:
            feature_emb = self.embedding_layer(inputs)
        for i in range(2, self.order + 1):
            order_i_out = self.high_order_interaction(feature_emb if self.reuse_embedding \
                                                      else self.embedding_layers[i - 2](inputs), order_i=i)
            y_pred += order_i_out
        if self.output_activation is not None:
            y_pred = self.output_activation(y_pred)
        return y_pred

    def high_order_interaction(self, feature_emb, order_i):
        if order_i == 2:
            interaction_out = self.inner_product_layer(feature_emb)
        elif order_i > 2:
            index = self.field_conjunction_dict[(order_i, 0)].to(self.device)
            hadamard_product = torch.index_select(feature_emb, 1, index)
            for k in range(1, order_i):
                index = self.field_conjunction_dict[(order_i, k)].to(self.device)
                hadamard_product = hadamard_product * torch.index_select(feature_emb, 1, index)
            interaction_out = hadamard_product.sum((1, 2)).view(-1, 1)
        return interaction_out

Example

In [ ]:
params = {'model_id': 'HOFM',
              'data_dir': '/content/data',
              'model_root': './checkpoints/',
              'learning_rate': 1e-3,
              'optimizer': 'adamw',
              'task': 'binary_classification',
              'loss': 'binary_crossentropy',
              'metrics': ['logloss', 'AUC'],
              'regularizer': 0,
              'order': 4,
              'embedding_dim': 10,
              'reuse_embedding': False,
              'batch_size': 64,
              'epochs': 3,
              'shuffle': True,
              'seed': 2019,
              'use_hdf5': True,
              'workers': 1,
              'verbose': 0}

In [ ]:
model = HOFM(ds.dataset.feature_map, **params)

In [ ]:
pl_trainer(model, ds, max_epochs=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name                | Type              | Params
----------------------------------------------------------
0 | embedding_layers    | ModuleList        | 14.3 K
1 | inner_product_layer | InnerProductLayer | 378   
2 | lr_layer            | LR_Layer          | 477   
3 | output_activation   | Sigmoid           | 0     
----------------------------------------------------------
14.8 K    Trainable params
378       Non-trainable params
15.1 K    Total params
0.061     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Metrics': {'AUC': tensor(1.), 'logloss': tensor(0.5169)}}
--------------------------------------------------------------------------------


[{'Test Metrics': {'AUC': tensor(1.), 'logloss': tensor(0.5169)}}]

## v2

In [ ]:
#export
import torch

from recohut.models.layers.common import FeaturesEmbedding, FeaturesLinear

In [ ]:
#export
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

class AnovaKernel(torch.nn.Module):

    def __init__(self, order, reduce_sum=True):
        super().__init__()
        self.order = order
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        batch_size, num_fields, embed_dim = x.shape
        a_prev = torch.ones((batch_size, num_fields + 1, embed_dim), dtype=torch.float).to(x.device)
        for t in range(self.order):
            a = torch.zeros((batch_size, num_fields + 1, embed_dim), dtype=torch.float).to(x.device)
            a[:, t+1:, :] += x[:, t:, :] * a_prev[:, t:-1, :]
            a = torch.cumsum(a, dim=1)
            a_prev = a
        if self.reduce_sum:
            return torch.sum(a[:, -1, :], dim=-1, keepdim=True)
        else:
            return a[:, -1, :]

class HOFM_v2(torch.nn.Module):
    """
    A pytorch implementation of Higher-Order Factorization Machines.
    Reference:
        M Blondel, et al. Higher-Order Factorization Machines, 2016.
    """

    def __init__(self, field_dims, order, embed_dim):
        super().__init__()
        if order < 1:
            raise ValueError(f'invalid order: {order}')
        self.order = order
        self.embed_dim = embed_dim
        self.linear = FeaturesLinear(field_dims)
        if order >= 2:
            self.embedding = FeaturesEmbedding(field_dims, embed_dim * (order - 1))
            self.fm = FactorizationMachine(reduce_sum=True)
        if order >= 3:
            self.kernels = torch.nn.ModuleList([
                AnovaKernel(order=i, reduce_sum=True) for i in range(3, order + 1)
            ])

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        y = self.linear(x).squeeze(1)
        if self.order >= 2:
            x = self.embedding(x)
            x_part = x[:, :, :self.embed_dim]
            y += self.fm(x_part).squeeze(1)
            for i in range(self.order - 2):
                x_part = x[:, :, (i + 1) * self.embed_dim: (i + 2) * self.embed_dim]
                y += self.kernels[i](x_part).squeeze(1)
        return torch.sigmoid(y)

> **References:-**
- M Blondel, et al. Higher-Order Factorization Machines, 2016.
- https://github.com/rixwew/pytorch-fm/blob/master/torchfm/model/hofm.py

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut